<a href="https://colab.research.google.com/github/Kimeunsim/datamining-with-R-Python-1/blob/main/xml%20import.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise Description

Exercise 6

A short description.


In [ ]:
EXERCISE = 6

# Group INFO

In [ ]:
# update the following values for my group
# open in browser the google drive folder and copy the ID part from the URL
# Example: if the url is https://drive.google.com/drive/folders/1Vntk6B89Gu0jEcfrPjJhuv7m9tbM6H0g
#.   then the ID is the last part: 1Vntk6B89Gu0jEcfrPjJhuv7m9tbM6H0g
GROUP_DRIVE_ID = "12Gn34Pnmgf52f91u9AeD75oKq-SYO2ww" #capital bc convention never change, constant through the course
GROUP_NUMBER = 2
GROUP_STUDENTS = [
    ("Kim Eunsim", "jiang201424@gmail.com"),
    ("Divya Thekkath", "divyadivakaranthekkath@gmail.com"),
    ("Jessica Pereira","jespereir.ds@gmail.com")

]

In [ ]:
import pathlib
import pandas as pd

# DO NOT CHANGE

In [ ]:
# prepare the data folders
# this code works for ALL members of the group
# when run from google colab AND when run on their own computers
# whenever you try to read/save data files, you should use
# pathlib.Path(folder, relative_path) - see example below
# where folder can be
# COURSE_PATH - for data shared for every group (readonly)
# GROUP_PATH - data shared for the group, (read-write)

try:
  from google.colab import drive
  DRIVE_PATH = "/content/drive"
  drive.mount(DRIVE_PATH, force_remount=True)
except ImportError:
  import platform
  if platform.system() == 'Windows':
    DRIVE_PATH = "G:"
  elif platform.system() == 'Darwin':
    DRIVE_PATH = "/Volumes/GoogleDrive"

COURSE_DRIVE_ID = "14N-wSVqibOfCuVUdNoaTw9sb-XmKRw86"

COURSE_PATH = DRIVE_PATH + "/.shortcut-targets-by-id/" + \
  COURSE_DRIVE_ID + "/dmshare/data"

GROUP_NAME = f"dm25_group{GROUP_NUMBER}"

GROUP_PATH = (DRIVE_PATH + "/.shortcut-targets-by-id/" +
  GROUP_DRIVE_ID + "/" + GROUP_NAME)


Mounted at /content/drive


In [ ]:
# test if the group folder is updated and mounted
if GROUP_DRIVE_ID == "1B2n50s0e_tahtrF0-1KPsseH4sybLjkQ" or GROUP_NAME == "dm25_group0":
  print("\x1b[31mYOU didn't update the GROUP info, see the Group INFO section","\x1b[0m")

if not pathlib.Path(GROUP_PATH).exists():
  print(f"\x1b[31mCheck if your GROUP Google Drive folder is: <{GROUP_NAME}>\x1b[0m")


# EXERCISE PATHS

In [ ]:
EX_SRC_PATH = COURSE_PATH + f"/ex{EXERCISE}"
EX_DST_PATH = GROUP_PATH + f"/output/ex{EXERCISE}"
pathlib.Path(EX_DST_PATH).mkdir(parents=True, exist_ok=True)

# # if you want to import a package not available on Google Colab
# # uncomment the following code
# try:
#   import feature_engine
# except ImportError:
#   # we install this package
#   !pip install feature_engine

# EXERCISE START

In [ ]:
filename="ex6/movies/0/000395.xml"

In [ ]:
#extract general movie information from the xml file
def get_movie(filename):
    df_movie=pd.read_xml(pathlib.Path(COURSE_PATH,filename),xpath="/doc")
    df_movie=df_movie[["docid","title","year","type","url"]]
    return df_movie


In [ ]:
#get genre of the movie
def get_genre_of_movie(filename, movie_id):
    df_genre=pd.read_xml(pathlib.Path(COURSE_PATH, filename), xpath="/doc/genres")
    df_genre["docid"]=movie_id
    return df_genre


In [ ]:
def get_cast_of_movie(filename,movie_id):
    df_cast=pd.read_xml(pathlib.Path(COURSE_PATH,filename), xpath="/doc/cast/credit")
    df_cast["docid"]=movie_id
    return df_cast

In [ ]:
#split the extraction into 3 functions
movie_df_list=[]
genre_df_list=[]
cast_df_list=[]

for filename in ["ex6/movies/0/000395.xml","ex6/movies/0/000051.xml"]:
  df_movie=get_movie(filename)
  movie_id=df_movie["docid"][0]
  df_genre=get_genre_of_movie(filename,movie_id)
  df_cast=get_cast_of_movie(filename,movie_id)

  movie_df_list.append(df_movie)
  genre_df_list.append(df_genre)
  cast_df_list.append(df_cast)

final_df_movie=pd.concat(movie_df_list)
final_df_genre=pd.concat(genre_df_list)
final_df_cast=pd.concat(cast_df_list)

In [ ]:
final_df_cast.head()

,actor,role,docid
0,"Anderson, C.E.",Ranger,395
1,"Aylesworth, Arthur",Innkeeper Flint,395
2,"Barrat, Robert",Humphrey Towne,395
3,"Bradley, Truman",Captain Ogden,395
4,"Brennan, Walter",'Hunk' Marriner,395


##3. check the data type and missing values, errors


In [ ]:
#data overview
for name, df in {
    "Movies": final_df_movie,
    "Genres": final_df_genre,
    "Casts": final_df_cast,

}.items():
    print(f"\n{name}->shape={df.shape}")
    print(df.dtypes)
    print("Missing values:\n",df.isna().sum())


Movies->shape=(2, 5)
docid     int64
title    object
year      int64
type     object
url      object
dtype: object
Missing values:
 docid    0
title    0
year     0
type     0
url      0
dtype: int64

Genres->shape=(2, 2)
genre    object
docid     int64
dtype: object
Missing values:
 genre    0
docid    0
dtype: int64

Casts->shape=(56, 3)
actor    object
role     object
docid     int64
dtype: object
Missing values:
 actor    0
role     0
docid    0
dtype: int64


##We can see no missing value and null values too

##4. Write into one CSV file

In [ ]:
combined_df=final_df_movie.merge(final_df_genre, on="docid",how="left")
combined_df=combined_df.merge(final_df_cast,on="docid",how="left")
combined_df.to_csv(pathlib.Path(EX_DST_PATH, "combined_df.csv"))

##5. Write the to an SQLite3 database.

In [ ]:
import sqlite3

In [ ]:
with sqlite3.connect(pathlib.Path(EX_DST_PATH, "combined_df.db")) as conn:
  combined_df.to_sql("movies_all_data",conn,if_exists="replace",index=False)

combined_df.to_sql

In [ ]:
import shutil

In [ ]:
shutil.copy(pathlib.Path(EX_DST_PATH, "combined_df.db"), pathlib.Path(EX_DST_PATH, "combined_df_copy.db"))

PosixPath('/content/drive/.shortcut-targets-by-id/12Gn34Pnmgf52f91u9AeD75oKq-SYO2ww/dm25_group2/output/ex6/combined_df_copy.db')